In [ ]:
import Pkg; Pkg.activate("..")
push!(LOAD_PATH, "../src/");

In [ ]:
# Uncomment to install dependencies
# Pkg.instantiate()

In [ ]:
using BenchmarkTools
using Combinatorics
using ProgressMeter
using Plots
using Random
using Santa

In [ ]:
cities = read_cities("../test/cities.csv");
path = read_path(cities, "../../scripts_julia/outputs/sub_perm_opt_1.516680222109079e6.csv");

In [ ]:
score(path)

### 2-opt

In [ ]:
import NearestNeighbors: KDTree, knn

In [ ]:
function optimize_2opt(init_path::Vector{City}, K::Int)
    path = copy(init_path)
    path_idxs = collect(1:length(init_path))
    
    cities_coords = map(c -> c.xy, init_path)
    kdtree = KDTree(cities_coords)

    @showprogress for i = 2:length(path)-1
        best_score, best_idx = 0, 0
        for j_init in knn(kdtree, path[i].xy, K)[1]
            j = path_idxs[j_init]
            ((j == 0) || (j <= i) || (j == length(path))) && continue
            s = score_2opt(path, i, j)
            if s < best_score
                best_score, best_idx = s, j
            end
        end
        if best_idx != 0
            reverse!(path, min(i, best_idx), max(i, best_idx))
            reverse!(path_idxs, min(i, best_idx), max(i, best_idx))
        end
    end
    
    path
end

In [ ]:
@time new_path = optimize_2opt(path, 15);

In [ ]:
score(path), score(new_path), verify!(new_path)

### 3-opt

In [ ]:
## Non-optimized version
# Optimize: limit memory copies, partial/diff scoring
function find_best_reconnection_3opt(path::Vector{City}, a::Int, b::Int, c::Int, idxs::Vector{Int}) # idxs is a HACK !
    α, β = a-1, c+1
    
    # Original
    original_score = score(path[α:β], start=α)
    best_route = path

    # Route 4
    new_route = vcat(view(path, α:a), view(path, b+1:c), view(path, a+1:b), view(path, c+1:β))
    new_score4 = score(new_route, start=α)

    # Route 1
    reverse!(new_route, 3, 3+c-(b+1))
    new_score1 = score(new_route, start=α)
    
    # Route 3
    reverse!(new_route, 3, 3+c-(b+1))
    reverse!(new_route, 4+c-(b+1), 4+b-(a+1))
    new_score3 = score(new_route, start=α)
    
    # Route 2
    new_route = vcat(view(path, α:a), reverse(view(path, a+1:b)), reverse(view(path, b+1:c)), view(path, c+1:β))
    new_score2 = score(new_route, start=α)

    best_score, best_idx = findmin([new_score1, new_score2, new_score3, new_score4, original_score])
    
    if best_idx == 2
        best_route = vcat(view(path, 1:a), reverse(view(path, a+1:b)), reverse(view(path, b+1:c)), view(path, c+1:length(path)))
        idxs = vcat(view(idxs, 1:a), reverse(view(idxs, a+1:b)), reverse(view(idxs, b+1:c)), view(idxs, c+1:length(idxs)))
        #reverse!(new_route, 3, 3+c-(b+1))
    elseif best_idx == 1
        best_route = vcat(view(path, 1:a), reverse(view(path, b+1:c)), view(path, a+1:b), view(path, c+1:length(path)))
        idxs = vcat(view(idxs, 1:a), reverse(view(idxs, b+1:c)), view(idxs, a+1:b), view(idxs, c+1:length(idxs)))
        #reverse!(new_route, 3, 3+c-(b+1))
        #reverse!(new_route, 4+c-(b+1), 4+b-(a+1))
    elseif best_idx == 4
        best_route = vcat(view(path, 1:a), view(path, b+1:c), view(path, a+1:b), view(path, c+1:length(path)))
        idxs = vcat(view(idxs, 1:a), view(idxs, b+1:c), view(idxs, a+1:b), view(idxs, c+1:length(idxs)))
        #reverse!(new_route, 4+c-(b+1), 4+b-(a+1))
    elseif best_idx == 3
        best_route = vcat(view(path, 1:a), view(path, b+1:c), reverse(view(path, a+1:b)), view(path, c+1:length(path)))
        idxs = vcat(view(idxs, 1:a), view(idxs, b+1:c), reverse(view(idxs, a+1:b)), view(idxs, c+1:length(idxs)))
    end

    best_score, best_route, idxs
end

In [ ]:
function optimize_3opt(init_path::Vector{City}, K::Int)
    path = copy(init_path)
    path_idxs = collect(1:length(init_path))

    cities_coords = map(c -> c.xy, init_path)
    kdtree = KDTree(cities_coords)
    
    best_score = score(path)

    @showprogress for i = 2:length(path)-1
        for j_init in knn(kdtree, path[i].xy, K)[1]
            # TODO: Changes nothing break
            j = path_idxs[j_init]
            ((j == 0) || (j <= i) || (j == length(path))) && continue
            
            for k_init in knn(kdtree, path[j].xy, K)[1]
                k = path_idxs[k_init]
                ((k == 0) || (k <= j) || (k == length(path))) && continue
                
                _, new_route, new_idxs = find_best_reconnection_3opt(path, i, j, k, path_idxs)
                new_score = score(new_route)
                if new_score < best_score
                    best_score = new_score
                    path = new_route
                    path_idxs = new_idxs
                    println(best_score)
                    verify!(path)
                end
            end
        end
    end
    
    path
end

In [ ]:
new_path_3opt = optimize_3opt(new_path, 15);

### Double Bridge

In [ ]:
# non-optimized implementation
function double_bridge(path::Vector{City})
    i = rand(1:length(path)-8) + 1
    ii = i+1
    
    j = rand(i+2:length(path)-6) + 1
    jj = j+1
    
    k = rand(j+2:length(path)-3) + 1
    kk = k+1
    
    l = rand(k+2:length(path)-2) + 1
    ll = l+1
    
    new_path = vcat(
        path[1:i],
        path[kk],
        path[kk+1:l],
        path[jj],
        path[jj+1:k],
        path[ii],
        path[ii+1:j],
        path[ll],
        path[ll+1:end]
    )
    
    verify!(new_path)
    new_path
end

In [ ]:
score(path)

In [ ]:
new_path = double_bridge(path)
score(new_path)